In [1]:
# Load MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
# start an interactive session, which allows you to interleave operations 
# which build a computation graph with ones that run the graph
sess = tf.InteractiveSession()


## Softmax Regression Model 

x and y_ are each a placeholder -- 
a value that we'll input when we ask TensorFlow to run a computation.

784 is the dimensionality of a single flattened MNIST image, and **None** indicates that the first dimension, corresponding to the **batch size, can be of any size**

shape argument is optional, it allows to catch bugs

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10]) #one-hot 10-dimensional vector indicating digit class
                    

The model parameters will be of type **Variable**. A Variable is a value that lives in TensorFlow's computation graph. It can be used and even modified by the computation.

In [4]:
# We initialize both W and b as tensors full of zeros
W = tf.Variable(tf.zeros([784,10])) # W is a 784x10 matrix (784 input features and 10 outputs) 
b = tf.Variable(tf.zeros([10])) # 10 outputs

Variables must be initialized withing a session. This step takes the initial values and assigns them to each Variable

In [5]:
sess.run(tf.initialize_all_variables())

Our regression model. Softmax per class for $Wx + b$ 

`tf.matmul` is flipped from when we multiplied $Wx$, as a small trick to deal with x being a 2D tensor with multiple inputs

In [7]:
y = tf.nn.softmax(tf.matmul(x,W) + b)


Cost function is the cross-entropy between the target and the model's prediction

In [22]:
print x.get_shape()
print W.get_shape()
print b.get_shape()
print y.get_shape()
# '?' of shape is equivalent to None whichindicates that 
# the first dimension, corresponding to the batch size, can be of any size

(?, 784)
(784, 10)
(10,)
(?, 10)


In [26]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1])) 
# tf.reduce_sum sums across all classes and tf.reduce_mean takes the average over these sums

In [27]:
print (-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1])).get_shape()
print cross_entropy.get_shape()

(?,)
()


### Train the Model

We will use steepest gradient descent, with a step length of 0.5, to descend the cross entropy.

The command below **adds new operations to the computation graph**. These operations included ones to *compute gradients*, *compute parameter update steps*, *and apply update steps to the parameters*.

In [28]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

The returned operation train_step, when run, will apply the gradient descent updates to the parameters. **Training the model can therefore be accomplished by repeatedly running train_step**.

In [29]:
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

### Evaluate the Model
`tf.argmax` gives you the index of the highest entry in a tensor along some axis

In [30]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
print correct_prediction

Tensor("Equal:0", shape=(?,), dtype=bool)


To determine what fraction are correct: cast to fp and take the mean

In [34]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print accuracy

Tensor("Mean_2:0", shape=(), dtype=float32)


Finally, we can evaluate our accuracy on the test data

In [35]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


# Build a Multilayer Convolutional Network

Two handy functions to initialize ReLU neurons with a slightly positive initial bias to avoid "dead neurons."

In [24]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size?

Here using the vanilla version. A stride of one and are zero padded so output is same size as input. Our pooling is max pooling over 2x2 blocks

To keep code cleaner, abstract into functions.

In [25]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

**First conv layer**: Convolution, followed by max pooling. Compute 32 features for each 5x5 patch. Its weight tensor will have a shape of **[5, 5, 1, 32]**. First two dimensions are **patch size**, next is **#input channels**, last is **#output channels**. We will also have a bias vector with a component for each output channel.

In [27]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, reshape x to a 4d tensor, second and third dimensions are width and height, final dimension is number of color channels.

In [28]:
x_image = tf.reshape(x, [-1,28,28,1])

We then convolve x_image with weight tensor, add bias, apply ReLU, and finally max pool.

In [29]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

 The second layer will have 64 features for each 5x5 patch.

In [31]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Now, image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons

We reshape the tensor into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU

In [32]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


**Dropout**

We create a **placeholder** for the probability that a neuron's output is kept during dropout. It **allows** us to turn dropout on during training, and **turn it off during testing**

`tf.nn.dropout` op **automatically handles** scaling neuron outputs in addition to masking them



In [33]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

** Readout: ** adding a softmax layer

In [35]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### Training
To train and evaluate, code is nearly identical to one layer SoftMax network. 

The differences are: 
1. Replace the SGD optimizer with ADAM optimizer
2. Add keep_prob to `feed_dict` of `.eval` and `.run`
3. Add logging to every 100th iteration in the training process.



In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) # ADAM opt.
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0: # add logging 
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0}) # add keep_prob=1
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5}) # add keep_prob=0.5

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})) # add keep_prob=0.5


step 0, training accuracy 0.1
step 100, training accuracy 0.74
step 200, training accuracy 0.88
step 300, training accuracy 0.88
